# RBX
## Övergripande arbetsgång
    * Importerar punkter från DGN-fil i arbetsmapp till den aktuella databasen.
      Hämtar in punkter till databasen och placerar dem under /RBX_clean
    * Beräknar avstånd från importerade punkter till fasledningar.
      Fasledningar har tidigare hämtats in från Wires.ipynb
    * I RBX_polygon.py skapas polygoner, grupperar punkter och hämtar de punkter som har minst avstånd till fas.
      Polygoner och punkter skrivs ut till shapefiler (.shp)
    * Polygoner och punkter hämtas in till geodatabasen, från shapefilerna, och lägger även till domäner, ledningsgata(nr)
      och littera.

## Sökvägar Magnus

In [1]:
import os
from pathlib import Path
prj_dir = Path(r"C:\Users\semnve\Desktop\RBX_och_SKB\RBX_2023_11_01")
LGs_info_name = 'Ledningar_RBX_2023_11_01.txt'
LGs_info_path = os.path.join(prj_dir, LGs_info_name)
#txt_file = 'Ledningar_Kantträd_2023-05-29.txt'
gdb_name = 'RBX_2023_11_01.gdb'
gdb = os.path.join(prj_dir, gdb_name)
serv_prj_dir = Path(r"Z:\Projekt\Analys_2023\ledningar")
cvd_LEDNINGSGATA = 'cvd_LEDNINGSGATA.txt'
cvd_LEDNINGSGATA_path = os.path.join(prj_dir, cvd_LEDNINGSGATA)
cvd_DATE = 'cvd_DATE_2023.txt'
cvd_DATE_path = os.path.join(prj_dir, cvd_DATE)
all_merged_name = "all_2023_11_01"
version = "2023_11_01"

## Sökvägar Karin

In [1]:
from pathlib import Path
import os
prj_dir = Path(r"C:\SVK_2023\pythonkörningar\2020-12-20")
LGs_info_path = Path(r"U:\Projekt\Data_2023\styrfiler\RBX_kantträd\2023-12-20.txt")
#txt_file = 'Ledningar_Kantträd_2023-05-29.txt'
gdb_name = 'RBX_2023_12_20.gdb'
gdb = os.path.join(prj_dir, gdb_name)
serv_prj_dir = Path(r"U:\Projekt\Analys_2023\ledningar")
cvd_LEDNINGSGATA_path = r"U:\Projekt\Data\Geodataunderlag_SVK\cvd_LEDNINGSGATA.txt"
cvd_DATE_path = r"C:\SVK_2023\pythonkörningar\skanningsdatum_2023.txt"
all_merged_name = "all_2023_12_20"
version = "2023_12_20"

# Merge SKB text files for all blocks of a powerline

In [2]:
import os
import sys
import glob
import fileinput
import SVK
import pandas as pd

In [ ]:
# Merge RBX txt files for all blocks of a powerline (LGXXX_Y) into one txt

#prj_dir = Path(r"Z:\Projekt\Analys\ledningar")

def combine_blocks(row):
    # TODO: Referera till kolumnnamn istället för kolumnindex för LG och line
    LG = row[0]
    line = row[1]
    line_dir = serv_prj_dir / LG / f"line_{line}"
    
    print(f"Doing {LG}_{line}")
    block_dir = os.path.join(line_dir, "RBX", "block")
    combined_blocks_path = os.path.join(line_dir, "RBX", "RBX_raw.txt")
    SVK.merge_blocks(block_dir, "*_pt*.txt", combined_blocks_path)

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
print(LGs_info)
LGs_info.apply(combine_blocks, axis=1)

## Denna kopierar över alla rbx_raw.txt till den temporära mappen på skrivbordet
### Är det nödvändigt? Jag tror att det som gör skillnad för hastigheten är om gdb:er / shapefiler ligger lokalt istället för på server. /Karin

In [3]:
import os
import filecmp
import shutil
import pandas as pd
df = pd.DataFrame()
to_dir = os.path.join(prj_dir, "ledningar")
from_dir = serv_prj_dir

#LGs_info_path = os.path.join(prj_dir, LGs_info_name)

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
testlist = []
for i, j in zip(LGs_info["LG"], LGs_info["line"]):
    testlist.append(f'{i}\\line_{j}\\RBX')

# print(testlist)
outlist_to_dir = []
for i in testlist:
    outlist_to_dir.append((f"{to_dir}\{i}\RBX_raw.txt").replace('\\\\', '\\'))

outlist_from_dir = []
for i in testlist:
    outlist_from_dir.append(
        (f"{from_dir}\{i}\RBX_raw.txt").replace('\\\\', '\\'))

for i, j in zip(outlist_from_dir, outlist_to_dir):
    if os.path.exists(j.replace('\\\\', '\\')):
        shutil.copyfile(i.replace('\\\\', '\\'), j.replace('\\\\', '\\'))
    else:
        folder1 = j.split("\\")[-4]
        folder2 = j.split("\\")[-3]
        folder3 = j.split("\\")[-2]
        os.makedirs(
            os.path.join(
                to_dir,
                folder1,
                folder2,
                folder3
            )
        )
        shutil.copyfile(i.replace('\\\\', '\\'), j.replace('\\\\', '\\'))

for i, j in zip(outlist_from_dir, outlist_to_dir):
    print(filecmp.cmp(i, j))


True
True


# Importera rensade RBX från dgn till gdb
Hämtar in punkter från DGN-filer till geodatabasen. <br>
DGN-filerna ska finnas som undermapp i projektmappen. <br>
Punkter som hämtas ska ligga i lager 20 i DGN-filerna.<br>
Om punkter ligger i annat lager kommer inga punkter att importeras. <br>
Ledningsgator och ledningar som behandlas utfrån från textfil där alla ledningar som ska köras finns med. <br>
Textfilen importeras och görs till en dataframe. <br>
I geodatabasen kommer punkterna placeras under clean_RBX_points.

In [ ]:
# Copy RBX from dgn to gdb feature class, one fc for each LGXXX_Y
import arcpy, os, glob, SVK
from pathlib import Path
import pandas as pd

dgn_dir = r"U:\Projekt\Analys_2023\DGN"
#dgn_dir = os.path.join(prj_dir, "DGN")
output_fd = os.path.join(gdb, "clean_RBX_points")

def sweref99TM():
    wkt = 'PROJCS["SWEREF99_TM",GEOGCS["GCS_SWEREF99",DATUM["D_SWEREF99",SPHEROID["GRS_1980",6378137,298.257222101]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",15],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["Meter",1]]'
    spatialref = arcpy.SpatialReference()
    spatialref.loadFromString(wkt)
    return(spatialref)

sr = sweref99TM()

def impdgn(powerline): 
    LG = powerline["LG"]
    line = powerline["line"]
    littera = powerline["Littera"]
    
    os.chdir(dgn_dir)
    print(LG)
    arcpy.env.overwriteOutput = True
    dgn = os.path.join(dgn_dir, LG+"_"+str(line)+".dgn") #
    print(dgn)

    output_fc = os.path.join(output_fd, LG+"_"+str(line)+"_RBX_clean") #var punkterna ska exporteras till

    print(os.path.join(output_fd, LG+"_"+str(line)+"_RBX_clean"))
    # Create a value table that will hold the input feature classes for Merge
    vTab = arcpy.ValueTable()
    print("vTab created")
    layername_RBX = dgn + '_Layer_LVL20'
    arcpy.MakeFeatureLayer_management(dgn + '/Point', layername_RBX, "\"Level\" = 20")
    print("MakeFeatureLayer complete")
    vTab.addRow([layername_RBX, 2])
    print("Row added to vTab")

    # Merge the CAD features into one feature class
    arcpy.env.workspace = output_fd
    arcpy.Merge_management(vTab, output_fc)
    print("vTab merged to output_fc")

    arcpy.Delete_management(layername_RBX)
    print("Layer deleted")

    arcpy.env.workspace = dgn_dir
 
LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs_info.apply(impdgn, axis=1)

LG012
U:\Projekt\Analys_2023\DGN\LG012_3.dgn
C:\SVK_2023\pythonkörningar\2020-12-20\RBX_2023_12_20.gdb\clean_RBX_points\LG012_3_RBX_clean
vTab created
MakeFeatureLayer complete
Row added to vTab
vTab merged to output_fc
Layer deleted
LG056
U:\Projekt\Analys_2023\DGN\LG056_1.dgn
C:\SVK_2023\pythonkörningar\2020-12-20\RBX_2023_12_20.gdb\clean_RBX_points\LG056_1_RBX_clean
vTab created
MakeFeatureLayer complete
Row added to vTab
vTab merged to output_fc
Layer deleted


0    None
1    None
dtype: object

# Near 3D - avstånd till fas för RBX
Beräknar avstånd från importerade punkter till aktuell ledning med hjälp av "Near3D_3d". <br>
Lägger till avstånd till punkterna i nya fält "AVSTAND_FAS" och "AVSTAND_HORISONTELLT" för respektive punkt. <br>
Skriptet utgår från tidigare nämnda textfil för vilka ledningsgator som ska behandlas.

In [ ]:
# This is where actual computations begin, previous steps are for arranging data
# This version of the script reads RBX points from a 3D point feature class in an ESRI file geodatabase (gdb)

import arcpy, os, glob, SVK
arcpy.CheckOutExtension('3D')
from pathlib import Path
import pandas as pd

fd_RBX = os.path.join(gdb, 'clean_RBX_points')
fd_wires = os.path.join(gdb, 'Wires')
arcpy.env.overwriteOutput = True

def dist2wire(fd_RBX, fc_name_RBX, fc_wires, radius):
    # Compute distance from RBX point to wire
    fc_RBX = os.path.join(fd_RBX, fc_name_RBX)
    arcpy.Near3D_3d(in_features=fc_RBX, near_features=fc_wires, search_radius=radius, location="NO_LOCATION", angle="NO_ANGLE", delta="NO_DELTA")
    print('done with Near3D '+fc_name_RBX)
    arcpy.DeleteField_management(in_table=fc_RBX, drop_field="NEAR_FID")
    arcpy.management.AlterField(in_table=fc_RBX, field="NEAR_DIST3", new_field_name="AVSTAND_FAS", new_field_alias="AVSTAND_FAS")
    arcpy.management.AlterField(in_table=fc_RBX, field="NEAR_DIST", new_field_name="AVSTAND_HORISONTELLT", new_field_alias="AVSTAND_HORISONTELLT")
    print('done with delete and alter field '+fc_name_RBX)
    
    
def RBX_3D_distance(powerline):
    #LG = powerline[0]
    #line = powerline[1]
    #littera = powerline[4]
    LG = powerline["LG"]
    line = powerline["line"]
    littera = powerline["Littera"]
    
    print(f"LG är {LG}, linje är {line}, littera är {littera}")
    
    fc_name_RBX = f"{LG}_{line}_RBX_clean"
    fc_name_wires = f"{LG}_{line}_fas"
    fc_wires = os.path.join(fd_wires, fc_name_wires)
    
    if arcpy.Exists(os.path.join(fd_RBX, fc_name_RBX)):
        dist2wire(fd_RBX, fc_name_RBX, fc_wires, 6)
        print(f"Distance to wire computed for RBX {LG}_{line}")
    else:
        print(f"{os.path.join(fd_RBX, fc_name_RBX)} does not exist")

# The file pointed at by LGs_info_path contains only the powerlines that should be
# processed. So this file has to be updated each time we want to process a new set of
# powerlines. This is unpractical - consider a user prompt where the user can select
# which powerlines to process

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs_info.apply(RBX_3D_distance, axis=1)


LG är LG012, linje är 3, littera är UL1 S1-3
done with Near3D LG012_3_RBX_clean
done with delete and alter field LG012_3_RBX_clean
Distance to wire computed for RBX LG012_3
LG är LG056, linje är 1, littera är RL8 S7
done with Near3D LG056_1_RBX_clean
done with delete and alter field LG056_1_RBX_clean
Distance to wire computed for RBX LG056_1


0    None
1    None
dtype: object

# (EXTERNT SKRIPT) Kör RBX_polygon.py
##  Innan nästa cell körs måste RBX_polygons.py köras.
RBX_polygon.py gör polygoner av RBX-punkterna och delar in dem i "farlighetsklass", sätter minsta avstånd till fas för varje polygon. <br>
Skriptet grupperar även alla RBX-punkter per polygon och beräknar närmsta punkt för varje polygon.

# Put RBX shape polygons in gdb

Skriver över geometrier från shapefiler till geodatabaser. <br>
Polygoner och alla närmaste punkter hämtas från shapefiler och skrivs sedan över till geodatabasen. <br>
Skriptet använder tidigare nämnda textfil för vilka ledningsgator som ska behandlas. <br>
Tre stycken olika lager kommer att skapas kommer att skapas i den valda geodatabasen.
    * RBX_polygons
    * RBX_closest_points
    * RBX_all_points


In [6]:
import arcpy, os, glob, SVK, shutil, datetime, time
from pathlib import Path
import pandas as pd
from datetime import date

RBX_shp_dir = os.path.join(prj_dir,"RBX_polygons")


# Paths to feature classes
RBX_polygons_fd = os.path.join(gdb,"RBX_polygons")                                                                                            # kolla även det ör som gäller vid utleverans om det är så att databasfilen ska kopieras över till utleverans
RBX_closest_points_fd = os.path.join(gdb,"RBX_closest_points")
RBX_all_points_fd = os.path.join(gdb,"RBX_all_points")

urspr = "SWECO"
ins_met = 20
matosak_plan = 1000
matosak_hojd = 1000
lev_dat = str(date.today())


def shp2gdb(powerline):
    LG = powerline[0]
    line = powerline[1]
    littera = powerline["Littera"]
    LG_code = cvd_LEDNINGSGATA[LG]
    LG_date = cvd_DATE[LG.join("_00" + str(line))]

        
    arcpy.env.overwriteOutput = True
    RBX_polygons_shp_path = os.path.join(RBX_shp_dir,f"{LG}_{line}_RBX.shp")
    RBX_closest_points_shp_path = os.path.join(RBX_shp_dir,f"{LG}_{line}_closest_points.shp")
    RBX_all_points_shp_path = os.path.join(RBX_shp_dir,f"{LG}_{line}_all_points.shp")
    RBX_polygons_fc = f"RBX_{LG}_{line}"
    RBX_closest_points_fc = f"RBX_closest_points_{LG}_{line}"
    RBX_all_points_fc = f"RBX_all_points_{LG}_{line}"    
    
    if os.path.exists(RBX_polygons_shp_path) or os.path.exists(RBX_closest_points_shp_path):
        ####################################################################################################################
        # Polygons
        arcpy.env.workspace = os.path.join(gdb,"RBX_polygons")
        # creating placeholders for polygons
        print(f"Converting RBX_polygon_{LG}_{line}")
        fc_polygon = f"RBX_polygon_{LG}_{line}"
        # Create a value table that will hold the input feature classes for Merge
        vTab_polygon = arcpy.ValueTable()
        print("vTab created")
        layer_polygon = "Layer_polygon"
        arcpy.Delete_management(layer_polygon)
        arcpy.MakeFeatureLayer_management(RBX_polygons_shp_path, layer_polygon)
        print("MakeFeatureLayer complete")
        vTab_polygon.addRow(layer_polygon)
        print("Row added to vTab_polygon")
        # Activating workspace for creating placeholders
        arcpy.env.workspace = RBX_polygons_fd
        #arcpy.env.outputCoordinateSystem = spatial_ref
        arcpy.env.outputCoordinateSystem = arcpy.SpatialReference('SWEREF99_TM')
        arcpy.env.overwriteOutput = True
        #merged_member_data = arcpy.SpatialReference(r'SWEREF99_TM')
        #arcpy.management.CreateFeatureclass(gdb_path, fc_polygon, arcpy.SpatialReference('SWEREF99_TM'))
        arcpy.Merge_management(vTab_polygon, fc_polygon)
        # Creating actual fields in the geodatabase and calculating values for
        # "LITTERA" and "LEDNINGSGATA".
        arcpy.management.AddField(fc_polygon, "LITTERA", "TEXT")
        arcpy.management.CalculateField(fc_polygon, "LITTERA", "".join(("'",littera,"'")))
        arcpy.management.AddField(fc_polygon, "LEDNINGSGATA", "LONG")
        arcpy.management.CalculateField(fc_polygon, "LEDNINGSGATA", int(LG_code))
        arcpy.management.AddField(fc_polygon, "RGDTM", "DATE")
        arcpy.management.CalculateField(fc_polygon, "RGDTM", "".join(("'",LG_date,"'")))
        arcpy.management.AddField(fc_polygon, "Ursprung", "TEXT")
        arcpy.management.CalculateField(fc_polygon, "Ursprung", "".join(("'", urspr, "'")))
        arcpy.management.AddField(fc_polygon, "Ursprung_Datum", "DATE")
        arcpy.management.CalculateField(fc_polygon, "Ursprung_Datum", "".join(("'", lev_dat, "'")))
        arcpy.management.AddField(fc_polygon, "Insamlingsmetod", "LONG")
        arcpy.management.CalculateField(fc_polygon, "Insamlingsmetod", int(ins_met))
        arcpy.management.AddField(fc_polygon, "Matosakerhet_Plan", "LONG")
        arcpy.management.CalculateField(fc_polygon, "Matosakerhet_Plan", int(matosak_plan))
        arcpy.management.AddField(fc_polygon, "Matosakerhet_Hojd", "LONG")
        arcpy.management.CalculateField(fc_polygon, "Matosakerhet_Hojd", int(matosak_hojd))
        arcpy.management.AddField(fc_polygon, "KLASS", "SHORT")
        
        # Moving (copying) values from KLASS_TEMP to KLASS column 
        arcpy.env.workspace = RBX_polygons_fd
        field = ['KLASS']
        cursor = arcpy.UpdateCursor(fc_polygon,field)

        s_field = ['KLASS_TEMP']
        s_cursor = arcpy.SearchCursor(fc_polygon,s_field)
        
        with arcpy.da.SearchCursor(fc_polygon,s_field) as s_cursor:
            with arcpy.da.UpdateCursor(fc_polygon,field) as cursor:  
                for s_row, row in zip(s_cursor, cursor):
                    row[0] = int(s_row[0])
                    cursor.updateRow(row)
        # Removing the 'temporary' layer originating from shp file.
        arcpy.management.DeleteField(fc_polygon, "KLASS_TEMP")
        
        dataset = RBX_polygons_fd
        sr = arcpy.SpatialReference("SWEREF99_TM")
        arcpy.DefineProjection_management(dataset, sr)
        # Assigning domains to fields
        arcpy.management.AssignDomainToField(fc_polygon, "LEDNINGSGATA", "cvd_LEDNINGSGATA")
        arcpy.management.AssignDomainToField(fc_polygon, "LITTERA", "cvd_LITTERA_LEDNING")
        arcpy.management.AssignDomainToField(fc_polygon, "Matosakerhet_Hojd", "cvd_MATOSAKERHET")
        arcpy.management.AssignDomainToField(fc_polygon, "Matosakerhet_Plan", "cvd_MATOSAKERHET")
        arcpy.management.AssignDomainToField(fc_polygon, "Insamlingsmetod", "cvd_INMATNINGSMETOD")

        ####################################################################################################################
        # Closest points
        arcpy.env.workspace = os.path.join(gdb,"RBX_closest_points")
        # creating placeholders for closest points
        print(f"Converting RBX_closest_points_{LG}_{line}")
        fc_closest_points = f"RBX_closest_points_{LG}_{line}"
        # Create a value table that will hold the input feature classes for Merge
        vTab_closest_points = arcpy.ValueTable()
        print("vTab created")
        layer_closest_points = "Layer_closest_points"
        arcpy.Delete_management(layer_closest_points)
        arcpy.MakeFeatureLayer_management(RBX_closest_points_shp_path, layer_closest_points)
        print("MakeFeatureLayer complete")
        vTab_closest_points.addRow(layer_closest_points)
        print("Row added to vTab_closest_points")
        # Activating workspace for creating placeholders
        arcpy.env.workspace = RBX_closest_points_fd
        arcpy.env.overwriteOutput = True
        arcpy.Merge_management(vTab_closest_points, fc_closest_points)
        # Creating actual fields in the geodatabase and calculating values for
        # "LITTERA" and "LEDNINGSGATA".
        arcpy.management.AddField(fc_closest_points, "LITTERA", "TEXT")
        arcpy.management.CalculateField(fc_closest_points, "LITTERA", "".join(("'",littera,"'")))
        arcpy.management.AddField(fc_closest_points, "LEDNINGSGATA", "LONG")
        arcpy.management.CalculateField(fc_closest_points, "LEDNINGSGATA", int(LG_code))
        arcpy.management.AddField(fc_closest_points, "RGDTM", "DATE")
        arcpy.management.CalculateField(fc_closest_points, "RGDTM", "".join(("'",LG_date,"'")))
        arcpy.management.AddField(fc_closest_points, "Ursprung", "TEXT")
        arcpy.management.CalculateField(fc_closest_points, "Ursprung", "".join(("'", urspr, "'")))
        arcpy.management.AddField(fc_closest_points, "Ursprung_Datum", "DATE")
        arcpy.management.CalculateField(fc_closest_points, "Ursprung_Datum", "".join(("'", lev_dat, "'")))
        arcpy.management.AddField(fc_closest_points, "Insamlingsmetod", "LONG")
        arcpy.management.CalculateField(fc_closest_points, "Insamlingsmetod", int(ins_met))
        arcpy.management.AddField(fc_closest_points, "Matosakerhet_Plan", "LONG")
        arcpy.management.CalculateField(fc_closest_points, "Matosakerhet_Plan", int(matosak_plan))
        arcpy.management.AddField(fc_closest_points, "Matosakerhet_Hojd", "LONG")
        arcpy.management.CalculateField(fc_closest_points, "Matosakerhet_Hojd", int(matosak_hojd))
        arcpy.management.AddField(fc_closest_points, "KLASS", "SHORT")

        # Moving (copying) values from KLASS_TEMP to KLASS column 
        arcpy.env.workspace = RBX_closest_points_fd
        field = ['KLASS']
        cursor = arcpy.UpdateCursor(fc_closest_points,field)

        s_field = ['KLASS_TEMP']
        s_cursor = arcpy.SearchCursor(fc_closest_points,s_field)
        
        with arcpy.da.SearchCursor(fc_closest_points,s_field) as s_cursor:
            with arcpy.da.UpdateCursor(fc_closest_points,field) as cursor:  
                for s_row, row in zip(s_cursor, cursor):
                    row[0] = int(s_row[0])
                    cursor.updateRow(row)
        # Removing the 'temporary' layer originating from shp file.
        arcpy.management.DeleteField(fc_closest_points, "KLASS_TEMP")
        
        dataset = RBX_closest_points_fd
        sr = arcpy.SpatialReference("SWEREF99_TM")
        arcpy.DefineProjection_management(dataset, sr)
        # Assigning domains to fields
        arcpy.management.AssignDomainToField(fc_closest_points, "LEDNINGSGATA", "cvd_LEDNINGSGATA")
        arcpy.management.AssignDomainToField(fc_closest_points, "LITTERA", "cvd_LITTERA_LEDNING")
        arcpy.management.AssignDomainToField(fc_closest_points, "Matosakerhet_Hojd", "cvd_MATOSAKERHET")
        arcpy.management.AssignDomainToField(fc_closest_points, "Matosakerhet_Plan", "cvd_MATOSAKERHET")
        arcpy.management.AssignDomainToField(fc_closest_points, "Insamlingsmetod", "cvd_INMATNINGSMETOD")
        
        ####################################################################################################################
        # All points
        arcpy.env.workspace = os.path.join(gdb,"RBX_all_points")
        # creating placeholders for closest points
        print(f"Converting RBX_all_points_{LG}_{line}")
        fc_all_points = f"RBX_all_points_{LG}_{line}"
        # Create a value table that will hold the input feature classes for Merge
        vTab_all_points = arcpy.ValueTable()
        print("vTab created")
        layer_all_points = "Layer_all_points"
        arcpy.Delete_management(layer_all_points)
        arcpy.MakeFeatureLayer_management(RBX_all_points_shp_path, layer_all_points)
        print("MakeFeatureLayer complete")
        vTab_all_points.addRow(layer_all_points)
        print("Row added to vTab_all_points")
        # Activating workspace for creating placeholders
        arcpy.env.workspace = RBX_all_points_fd
        arcpy.env.overwriteOutput = True
        arcpy.Merge_management(vTab_all_points, fc_all_points)
        # Creating actual fields in the geodatabase and calculating values for
        # "LITTERA" and "LEDNINGSGATA".
        arcpy.management.AddField(fc_all_points, "LITTERA", "TEXT")
        arcpy.management.CalculateField(fc_all_points, "LITTERA", "".join(("'",littera,"'")))
        arcpy.management.AddField(fc_all_points, "LEDNINGSGATA", "LONG")
        arcpy.management.CalculateField(fc_all_points, "LEDNINGSGATA", int(LG_code))
        arcpy.management.AddField(fc_all_points, "RGDTM", "DATE")
        arcpy.management.CalculateField(fc_all_points, "RGDTM", "".join(("'",LG_date,"'")))
        arcpy.management.AddField(fc_all_points, "Ursprung", "TEXT")
        arcpy.management.CalculateField(fc_all_points, "Ursprung", "".join(("'", urspr, "'")))
        arcpy.management.AddField(fc_all_points, "Ursprung_Datum", "DATE")
        arcpy.management.CalculateField(fc_all_points, "Ursprung_Datum", "".join(("'", lev_dat, "'")))
        arcpy.management.AddField(fc_all_points, "Insamlingsmetod", "LONG")
        arcpy.management.CalculateField(fc_all_points, "Insamlingsmetod", int(ins_met))
        arcpy.management.AddField(fc_all_points, "Matosakerhet_Plan", "LONG")
        arcpy.management.CalculateField(fc_all_points, "Matosakerhet_Plan", int(matosak_plan))
        arcpy.management.AddField(fc_all_points, "Matosakerhet_Hojd", "LONG")
        arcpy.management.CalculateField(fc_all_points, "Matosakerhet_Hojd", int(matosak_hojd))
        arcpy.management.AddField(fc_all_points, "KLASS", "SHORT")
        
        # Moving (copying) values from KLASS_TEMP to KLASS column 
        arcpy.env.workspace = RBX_all_points_fd
        field = ['KLASS']
        cursor = arcpy.UpdateCursor(fc_all_points,field)

        s_field = ['KLASS_TEMP']
        s_cursor = arcpy.SearchCursor(fc_all_points,s_field)
        
        with arcpy.da.SearchCursor(fc_all_points,s_field) as s_cursor:
            with arcpy.da.UpdateCursor(fc_all_points,field) as cursor:  
                for s_row, row in zip(s_cursor, cursor):
                    row[0] = int(s_row[0])
                    cursor.updateRow(row)
        # Removing the 'temporary' layer originating from shp file.
        arcpy.management.DeleteField(fc_all_points, "KLASS_TEMP")
        
        dataset = RBX_all_points_fd
        sr = arcpy.SpatialReference("SWEREF99_TM")
        arcpy.DefineProjection_management(dataset, sr)
        # Assigning domains to fields
        arcpy.management.AssignDomainToField(fc_all_points, "LEDNINGSGATA", "cvd_LEDNINGSGATA")
        arcpy.management.AssignDomainToField(fc_all_points, "LITTERA", "cvd_LITTERA_LEDNING")
        arcpy.management.AssignDomainToField(fc_all_points, "Matosakerhet_Hojd", "cvd_MATOSAKERHET")
        arcpy.management.AssignDomainToField(fc_all_points, "Matosakerhet_Plan", "cvd_MATOSAKERHET")
        arcpy.management.AssignDomainToField(fc_all_points, "Insamlingsmetod", "cvd_INMATNINGSMETOD")
    else:
        print(f"No RBX for {LG}_{line}")
    
LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
df_cvd_LEDNINGSGATA = pd.read_csv(cvd_LEDNINGSGATA_path, sep="\t", header=0)
df_cvd_DATE = pd.read_csv(cvd_DATE_path, sep="\t", header=0)
cvd_LEDNINGSGATA = {df_cvd_LEDNINGSGATA.LG[i]: df_cvd_LEDNINGSGATA.Code[i] for i in range(len(df_cvd_LEDNINGSGATA))}
cvd_DATE = {df_cvd_DATE.LG[i].join("_00" + str(df_cvd_DATE.nr[i])): df_cvd_DATE.Datum[i] for i in range(len(df_cvd_DATE))}

LGs_info.apply(shp2gdb, axis=1)

No RBX for LG040_1
No RBX for LG040_2
Converting RBX_polygon_LG040_3
vTab created
MakeFeatureLayer complete
Row added to vTab_polygon
Converting RBX_closest_points_LG040_3
vTab created
MakeFeatureLayer complete
Row added to vTab_closest_points
Converting RBX_all_points_LG040_3
vTab created
MakeFeatureLayer complete
Row added to vTab_all_points
Converting RBX_polygon_LG040_4
vTab created
MakeFeatureLayer complete
Row added to vTab_polygon
Converting RBX_closest_points_LG040_4
vTab created
MakeFeatureLayer complete
Row added to vTab_closest_points
Converting RBX_all_points_LG040_4
vTab created
MakeFeatureLayer complete
Row added to vTab_all_points
Converting RBX_polygon_LG040_5
vTab created
MakeFeatureLayer complete
Row added to vTab_polygon
Converting RBX_closest_points_LG040_5
vTab created
MakeFeatureLayer complete
Row added to vTab_closest_points
Converting RBX_all_points_LG040_5
vTab created
MakeFeatureLayer complete
Row added to vTab_all_points
No RBX for LG040_6
Converting RBX_poly

0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
8    None
9    None
dtype: object

# Komplettering lagernamn
Eftersom shapefiler inte kan fältnamn längre än 10 symboler, kompletter detta skript lagernamnen enligt<br>
SVKs manual.<br>
Ändrar även namn på dz till DELTA_HOJD, vilken kan bytas ut i det externa skriptet när tid ges.

In [9]:
import arcpy, os, glob, SVK, shutil
from pathlib import Path
import pandas as pd

RBX_shp_dir = os.path.join(prj_dir,"RBX_polygons")
RBX_polygons_fd = os.path.join(gdb,"RBX_polygons")                                                                                            
RBX_closest_points_fd = os.path.join(gdb,"RBX_closest_points")
RBX_all_points_fd = os.path.join(gdb,"RBX_all_points")

def shp2gdb(powerline):
    LG = powerline[0]
    line = powerline[1]
    
    RBX_polygons_shp_path = os.path.join(RBX_shp_dir,f"{LG}_{line}_RBX.shp")
    RBX_closest_points_shp_path = os.path.join(RBX_shp_dir,f"{LG}_{line}_closest_points.shp")
    RBX_all_points_shp_path = os.path.join(RBX_shp_dir,f"{LG}_{line}_all_points.shp")
    RBX_polygons_fc = f"RBX_polygon_{LG}_{line}"
    RBX_closest_points_fc = f"RBX_closest_points_{LG}_{line}"
    RBX_all_points_fc = f"RBX_all_points_{LG}_{line}"
    ch_fd_lst = [RBX_polygons_fd, RBX_closest_points_fd, RBX_all_points_fd]
    ch_fc_lst = [RBX_polygons_fc, RBX_closest_points_fc,RBX_all_points_fc]
    
    if os.path.exists(RBX_polygons_shp_path):
        for fd, fc in zip(ch_fd_lst, ch_fc_lst):
            print(f"Altering fields for {LG}_{line}")
            path_joined = os.path.join(fd, fc)
            field_names = [f.name for f in arcpy.ListFields(path_joined)]
            if "AVSTAND_HO" in field_names: 
                print(f"Altering field for {LG}_{line}")
                arcpy.management.AlterField(
                    path_joined,
                    "AVSTAND_HO",
                    "AVSTAND_HORISONTELLT",
                    "AVSTAND_HORISONTELLT")
            if "AVSTAND_FA" in field_names:    
                arcpy.management.AlterField(
                    path_joined,
                    "AVSTAND_FA",
                    "AVSTAND_FAS",
                    "AVSTAND_FAS")
            if "dz" in field_names:    
                arcpy.management.AlterField(
                    path_joined,
                    "dz",
                    "DELTA_HOJD",
                    "DELTA_HOJD")
    else:
        print(f"No RBX for {LG}_{line}")
    
LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs_info.apply(shp2gdb, axis=1)

Altering fields for LG026_1
Altering field for LG026_1
Altering fields for LG026_1
Altering field for LG026_1
Altering fields for LG026_1
Altering field for LG026_1
Altering fields for LG026_2
Altering field for LG026_2
Altering fields for LG026_2
Altering field for LG026_2
Altering fields for LG026_2
Altering field for LG026_2
No RBX for LG026_3
Altering fields for LG026_4
Altering field for LG026_4
Altering fields for LG026_4
Altering field for LG026_4
Altering fields for LG026_4
Altering field for LG026_4
Altering fields for LG026_6
Altering field for LG026_6
Altering fields for LG026_6
Altering field for LG026_6
Altering fields for LG026_6
Altering field for LG026_6
No RBX for LG040_1
No RBX for LG040_2
No RBX for LG040_3
No RBX for LG040_4
No RBX for LG040_5
No RBX for LG040_6
No RBX for LG040_7
No RBX for LG040_8
Altering fields for LG041_1
Altering field for LG041_1
Altering fields for LG041_1
Altering field for LG041_1
Altering fields for LG041_1
Altering field for LG041_1
Alte

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
27    None
28    None
dtype: object

## Städar bort feature classes som inte behövs för utleverans

In [10]:
import arcpy
 
# Set environment settings
arcpy.env.workspace = gdb

# Set local variables
features = [
    "clean_RBX_points",
    "mittfas",
    "Wires",
    "Gator2021",
    "SKB_template"]

for feature in features:
    arcpy.management.Delete(os.path.join(gdb, feature))